[slides](https://docs.google.com/presentation/d/19K9nVjuSOCrZGM6lmFeEEarTm2xZwDSiZEIzf-Ywr5o/edit?usp=sharing)

[python-ecdsa docs](https://github.com/warner/python-ecdsa)

# Signing our First Message with ECDSA

In [22]:
from ecdsa import SigningKey, SECP256k1

# SigningKey = abstraction for private keys
# SECP256k1 = refers to the parameters of the elliptic curve used in
# Bitcoin,y2 = x3 + 7 over real numbers, finite field (a.k.a. Galois field).

private_key = SigningKey.generate(curve=SECP256k1)
public_key = private_key.get_verifying_key()

message = b"Stay humble, stack sats."
signature = private_key.sign(message)

In [23]:
public_key.verify(signature, message)

True

In [24]:
public_key.verify(signature, b"getting old")

BadSignatureError: 

# Defining ECDSACoin

* A coin is just a list of transfers, just like with PNGCoin. 
    * Where transfers were photographs of signatures in PNGCoin, they are ECDSA digital signatures in ECDSACoin
* The `public_key` in the last transfer is who owns the coin
* To spend the coin append a new transfer. Use the public key of the person you are sending to, and sign it using your private key.

In [25]:
class Transfer:
    
    def __init__(self, signature, public_key):
        #attributes of a Transfer
        self.signature = signature   # authentication of the owner of coin
        self.public_key = public_key # identification of the recipient of coin
        
class ECDSACoin:
    
    def __init__(self, transfers):
        self.transfers = transfers  # transfer counts as ECDSACoin

In [26]:
# The usual suspects ... 
# SECP256k1 is a detail about the "magical multiplication" used under the covers

bank_private_key = SigningKey.generate(curve=SECP256k1)
bob_private_key = SigningKey.generate(curve=SECP256k1)
alice_private_key = SigningKey.generate(curve=SECP256k1)

bank_public_key = bank_private_key.get_verifying_key()
bob_public_key = bob_private_key.get_verifying_key()
alice_public_key = alice_private_key.get_verifying_key()

In [27]:
from utils import serialize

#bank is always the issuer of coins
def issue(public_key):
    
    message = serialize(public_key)  # we need to convert data to bytes.
    # for now our message only says we are issuing coins to this public key.
    signature = bank_private_key.sign(message)
    
    transfer = Transfer(       # creating a transfer
        signature=signature,   # message signed by bank's private key
        public_key=public_key, # public_key of the recipient
    )
    
    coin = ECDSACoin([transfer]) # using that transfer as the only contents of the list of transfers in a new ECDSACoin instance
    return coin

# Validating the First Transfer

In [28]:
def validate(coin):
    transfer = coin.transfers[0]  # hardcoded; for now we only consider we have 1 transfer from the bank.
    message = serialize(transfer.public_key) # message is the public key of the transfer.
    bank_public_key.verify(transfer.signature, message) # checking with the signature of the transfer.    

In [29]:
alice_coin = issue(alice_public_key)

validate(alice_coin)

In [30]:
# creating fraudulent issue, bob issuing coins to himself and trying to validate that

message = serialize(bob_public_key)
signature = bob_private_key.sign(message)
transfer = Transfer(
    signature=signature,
    public_key=bob_public_key,
)

alt_coin = ECDSACoin([transfer])

In [31]:
# demonstrating catching an error and handling it
from ecdsa import BadSignatureError

try:
    validate(alt_coin)              # try running this code
except BadSignatureError:           # if you get this
    print("Bad signature detected") # do this

Bad signature detected


# Validating Subsequent Transfers

In [32]:
# here is where we create a link between transfers by checking previous signatures while making new transfers.

def transfer_message(previous_signature, next_owner_public_key):
    return serialize({  # why are we returning a dict?
        "previous_signature": previous_signature,       # sig of the intended coin to be spent
        "next_owner_public_key": next_owner_public_key, # public key of next owner
    })

def validate(coin):
    # Check the first transfer
    transfer = coin.transfers[0]
    message = serialize(transfer.public_key)
    bank_public_key.verify(transfer.signature, message)
    
    # Check the rest of coin transfers
    previous_transfer = coin.transfer[0]
    for next_transfer in coin.transfers[1:]:
        message = transfer_message(previous_transfer.signature, next_transfer.public_key) # previous_transfer.signature = sig of the UTXO, next_transfer.public_key = intended owner/recipient
        previous_transfer.public_key.verify( # taking this to verify signature in next transfer
            transfer.signature,              # this = sig of next transfer
            message,                         # and that, that sig is for a particular message => this.
        )
        # 

In [37]:
# helper func: given a coin who owns it? always returns the pub key of last transfer(current owner)

def get_owner(coin):
    database = {
        bob_public_key: "Bob",
        alice_public_key: "Alice",
        bank_public_key: "Bank",
    }
    public_key = coin.transfers[-1].public_key
    return database[public_key]

In [38]:
# going through the process of minting a coin and transfering ownership of coin
coin = issue(alice_public_key)

print("This coin is owned by", get_owner(coin))

# message = transfer_message(coin.transfers[0].signature, bob_public_key) # creating the message to transfer coin from alice to bob

# alice_to_bob = Transfer(
#     signature=alice_private_key.sign(message), # signing the transfer message with alice's private key.
#     public_key=bob_public_key,
# )
# coin.transfers.append(alice_to_bob) # adding the transfer to the list of transfers.

# print("This coin is owned by", get_owner(coin))

TypeError: unhashable type: 'VerifyingKey'

# Serialization

# The Finished Product

[ecdsacoin.py](ecdsacoin.py)

In [21]:
import ecdsacoin

coin = ecdsacoin.issue(alice_public_key)
coin.validate()

alice_to_bob = Transfer(
    signature=alice_private_key.sign(transfer_message(coin.transfers[-1].signature, bob_public_key)),
    public_key=bob_public_key,
)

coin.transfers.append(alice_to_bob)
coin.validate()